In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score
X_train = pd.read_csv("data/x_train.txt",header=None,sep=" ")
y_train = pd.read_csv("data/y_train.txt",header=None,sep=" ")
def eval_proba(probas,y,n_features, num_target = 1000):
    sorted_probas = np.sort(probas)[::-1]  
    threshold = sorted_probas[num_target - 1]

    y_pred = (probas >= threshold).astype(int)
    gain = 10*precision_score(y, y_pred)*num_target
    return np.round(gain - 200*n_features)

In [ ]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from sklearn.svm import SVC
from sklearn.metrics import precision_score, f1_score, accuracy_score
import pandas as pd
import numpy as np

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'n_features': [1, 2, 3, 4, 5, 6]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
best_gain = -np.inf
best_params = None
history = []

for params in ParameterGrid(param_grid):
    cv_gains = []
    cv_precisions = []
    cv_f1s = []
    cv_accuracies = []
    
    for train_idx, val_idx in cv.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx, 0], y_train.iloc[val_idx, 0]

        mi = mutual_info_classif(X_tr, y_tr, random_state=42)
        top_idx = np.argsort(mi)[::-1][:params['n_features']]
        
        clf = SVC(
            C=params['C'],
            kernel=params['kernel'],
            gamma=params['gamma'],
            probability=True,    
            random_state=42
        )
        clf.fit(X_tr.iloc[:, top_idx], y_tr)
        
        probas = clf.predict_proba(X_val.iloc[:, top_idx])[:, 1]
        y_pred = (probas >= 0.5).astype(int)
        
        gain = eval_proba(probas, y_val.values, params['n_features'], num_target=200)
        precision = precision_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        accuracy = accuracy_score(y_val, y_pred)
        
        cv_gains.append(gain)
        cv_precisions.append(precision)
        cv_f1s.append(f1)
        cv_accuracies.append(accuracy)

    avg_gain = np.mean(cv_gains)
    avg_precision = np.mean(cv_precisions)
    avg_f1 = np.mean(cv_f1s)
    avg_accuracy = np.mean(cv_accuracies)
    
    print(f"Params: {params}, gain: {avg_gain:.2f}, precision: {avg_precision:.3f}, f1: {avg_f1:.3f}, accuracy: {avg_accuracy:.3f}")

    history.append({**params,
                    "gain": avg_gain,
                    "precision": avg_precision,
                    "f1": avg_f1,
                    "accuracy": avg_accuracy})

    if avg_gain > best_gain:
        best_gain = avg_gain
        best_params = params

print(f"Best parameters: {best_params}")
print(f"Max gain: {best_gain}")

history_df = pd.DataFrame(history)
display(history_df.sort_values("gain", ascending=False).head(10))


Params: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear', 'n_features': 1}, gain: 1250.00, precision: 0.714, f1: 0.690, accuracy: 0.707
Params: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear', 'n_features': 2}, gain: 1048.00, precision: 0.714, f1: 0.691, accuracy: 0.708
Params: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear', 'n_features': 3}, gain: 854.00, precision: 0.715, f1: 0.693, accuracy: 0.709
Params: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear', 'n_features': 4}, gain: 656.00, precision: 0.724, f1: 0.703, accuracy: 0.718
Params: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear', 'n_features': 5}, gain: 456.00, precision: 0.725, f1: 0.702, accuracy: 0.718
Params: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear', 'n_features': 6}, gain: 260.00, precision: 0.723, f1: 0.703, accuracy: 0.718
Params: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'n_features': 1}, gain: 1260.00, precision: 0.704, f1: 0.698, accuracy: 0.707
Params: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'n_featu

,C,gamma,kernel,n_features,gain,precision,f1,accuracy
54,10.0,scale,rbf,1,1310.0,0.702452,0.697411,0.7064
30,1.0,scale,rbf,1,1296.0,0.704226,0.697875,0.7074
6,0.1,scale,rbf,1,1260.0,0.703719,0.698250,0.7074
18,0.1,auto,rbf,1,1252.0,0.702240,0.697272,0.7062
24,1.0,scale,linear,1,1250.0,0.713980,0.690139,0.7070
0,0.1,scale,linear,1,1250.0,0.714296,0.690285,0.7072
60,10.0,auto,linear,1,1250.0,0.715110,0.690435,0.7076
36,1.0,auto,linear,1,1250.0,0.713980,0.690139,0.7070
48,10.0,scale,linear,1,1250.0,0.715110,0.690435,0.7076
12,0.1,auto,linear,1,1250.0,0.714296,0.690285,0.7072
